In [14]:
import requests
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

with open('../.api_auth') as f:
    (api_login, api_pass) = [line.rstrip() for line in f]

In [16]:
ids = []
leads = []
texts = []

date_range = pd.date_range(start="2023-04-01", end="2023-04-30").strftime('%d.%m.%Y').to_list()
for date in tqdm(date_range):
    r1 = requests.get(f'https://api.sta.si/news/sl/{date}', auth=(api_login, api_pass))
    articles_id = eval(r1.content)

    for id in articles_id:
        r2 = requests.get(f'https://api.sta.si/news/sta/{id}', auth=(api_login, api_pass))
        ids.append(id)
        leads.append(r2.json().get('lede', ''))
        texts.append(r2.json().get('text', ''))


100%|██████████| 30/30 [05:58<00:00, 11.95s/it]


In [25]:
dataset = pd.DataFrame({'id': ids, 'lead': leads, 'text': texts})
dataset.to_json(f'../data/raw_articles/04_2023.jsonl', lines=True, orient='records')